# 제품 이상여부 판별 프로젝트

In [1]:
import numpy as np
import pandas as pd

## 모델링

### 데이터 구분

In [2]:
THRESHOLD = 0.3
RANDOM_STATE = 110

# csv 불러오기
train_data = pd.read_csv('./data/train_data_0827.csv')
test_data = pd.read_csv('./data/test_data_0827.csv')

In [3]:
# dam, fill1, fill2 공통 변수
var_dam_fill = [
    'Receip_No_encoded',
    'Equipment_same_num',
    'PalletID_Collect_Result_encoded',
    'Production_Qty_Collect_Result',
    'WorkMode Collect Result'
]

In [4]:
# 전체 공통 변수
### train
var_all_train = [
    'target',
    'model_suffix_encoded',
    'cleaned_workorder_encoded'
]

### test
var_all_test = [
    'Set ID',
    'target',
    'model_suffix_encoded',
    'cleaned_workorder_encoded'
]

In [5]:
# '_Dam'을 포함하는 변수 선택
dam_variables = [var for var in train_data.columns if '_Dam' in var]

# train
final_columns_train = var_dam_fill + var_all_train + dam_variables
train_data_dam = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + dam_variables
test_data_dam = test_data[final_columns_test]

In [6]:
# '_Fill1'을 포함하는 변수 선택
fill1_variables = [var for var in train_data.columns if '_Fill1' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill1_variables
train_data_fill1 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill1_variables
test_data_fill1 = test_data[final_columns_test]

In [7]:
# '_Fill2'을 포함하는 변수 선택
fill2_variables = [var for var in train_data.columns if '_Fill2' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill2_variables
train_data_fill2 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill2_variables
test_data_fill2 = test_data[final_columns_test]

In [8]:
# '_AutoClave'을 포함하는 변수 선택
autoclave_variables = [var for var in train_data.columns if '_AutoClave' in var]

# train
final_columns_train = var_all_train + autoclave_variables
train_data_autoclave = train_data[final_columns_train]

# test 
final_columns_test = var_all_test + autoclave_variables
test_data_autoclave = test_data[final_columns_test]


In [9]:
# 각 DataFrame의 칼럼 수 계산
num_columns_train_data = train_data.shape[1]
num_columns_train_data_dam = train_data_dam.shape[1]
num_columns_train_data_autoclave = train_data_autoclave.shape[1]
num_columns_train_data_fill1 = train_data_fill1.shape[1]
num_columns_train_data_fill2 = train_data_fill2.shape[1]

num_columns_test_data = test_data.shape[1]
num_columns_test_data_dam = test_data_dam.shape[1]
num_columns_test_data_autoclave = test_data_autoclave.shape[1]
num_columns_test_data_fill1 = test_data_fill1.shape[1]
num_columns_test_data_fill2 = test_data_fill2.shape[1]

# 각 DataFrame의 칼럼 수 출력
print("----train data-----")
print(f"train_data DataFrame의 칼럼 수: {num_columns_train_data}")
print(f"train_data_dam DataFrame의 칼럼 수: {num_columns_train_data_dam}")
print(f"train_data_autoclave DataFrame의 칼럼 수: {num_columns_train_data_autoclave}")
print(f"train_data_fill1 DataFrame의 칼럼 수: {num_columns_train_data_fill1}")
print(f"train_data_fill2 DataFrame의 칼럼 수: {num_columns_train_data_fill2}")
print("----test data-----")
print(f"test_data DataFrame의 칼럼 수: {num_columns_test_data}")
print(f"test_data_dam DataFrame의 칼럼 수: {num_columns_test_data_dam}")
print(f"test_data_autoclave DataFrame의 칼럼 수: {num_columns_test_data_autoclave}")
print(f"test_data_fill1 DataFrame의 칼럼 수: {num_columns_test_data_fill1}")
print(f"test_data_fill2 DataFrame의 칼럼 수: {num_columns_test_data_fill2}")

----train data-----
train_data DataFrame의 칼럼 수: 40
train_data_dam DataFrame의 칼럼 수: 23
train_data_autoclave DataFrame의 칼럼 수: 8
train_data_fill1 DataFrame의 칼럼 수: 14
train_data_fill2 DataFrame의 칼럼 수: 14
----test data-----
test_data DataFrame의 칼럼 수: 41
test_data_dam DataFrame의 칼럼 수: 24
test_data_autoclave DataFrame의 칼럼 수: 9
test_data_fill1 DataFrame의 칼럼 수: 15
test_data_fill2 DataFrame의 칼럼 수: 15


모델 정의

In [10]:
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.ensemble import VotingClassifier

# 스레드홀드 설정
THRESHOLD = 0.3

# 모델 설정 및 하이퍼파라미터
models = {
    'et': ExtraTreesClassifier(),
    'rf': RandomForestClassifier(),
    'cat': CatBoostClassifier(),
    'lgbm': LGBMClassifier(),
    'xgb': XGBClassifier(),
    'dt': DecisionTreeClassifier(),
    'ada': AdaBoostClassifier()
}

def train_and_evaluate_model(model_name, data, **params):
    if model_name not in models:
        print(f"{model_name}은(는) 지원되지 않는 모델입니다.")
        return
    
    # 데이터셋 분할
    x_train, x_val, y_train, y_val = train_test_split(
        data.drop("target", axis=1),
        data["target"].map({'Normal': 0, 'AbNormal': 1}),
        test_size=0.2,
        shuffle=True,
        random_state=RANDOM_STATE,
    )

    # 모델 선택
    model = models[model_name].__class__()  # 새로운 모델 인스턴스 생성

    # 하이퍼파라미터 설정
    model.set_params(**params)

    # 모델 학습
    model.fit(x_train, y_train)

    # 데이터 이름을 자동으로 추출하기 위한 래퍼 함수
    data_name = [name for name in globals() if globals()[name] is data][0]

    # 예측
    y_val_pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
    y_val_pred = (y_val_pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측

    # 평가지표 계산
    f1 = f1_score(y_val, y_val_pred, average="binary")
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred, zero_division=0)
    recall = recall_score(y_val, y_val_pred)
    conf_matrix = confusion_matrix(y_val, y_val_pred)
    
    # 결과 출력
    print(f'{model_name} 모델이 {data_name} 데이터로 학습한 결과:')
    print(f'F1 Score: {f1}')
    print('---')
    print('Confusion Matrix:')
    print(conf_matrix)
    print('---')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print('\n')

    return model  # 학습된 모델 반환

def fit_all_train_data_function(model_name, data, **params):
    if model_name not in models:
        print(f"{model_name}은(는) 지원되지 않는 모델입니다.")
        return None  # 지원되지 않는 모델일 경우 None 반환
    
    # 모델 선택
    model = models[model_name].__class__()  # 새로운 모델 인스턴스 생성

    # 하이퍼파라미터 설정
    model.set_params(**params)

    # 모델 학습
    model.fit(data.drop("target", axis=1), data["target"].map({'Normal': 0, 'AbNormal': 1}))

    # 데이터 이름을 자동으로 추출하기 위한 래퍼 함수
    data_name = [name for name in globals() if globals()[name] is data][0]

    print(f'{model_name} 모델이 {data_name} 데이터로 학습 완료')
    return model  # 학습된 모델 반환

def voting_function(data, estimators, voting='hard', threshold=0.5):
    # 데이터셋 분할 # voting='hard'일 경우 threshold는 사용되지 않음
    x_train, x_val, y_train, y_val = train_test_split(
        data.drop("target", axis=1),
        data["target"].map({'Normal': 0, 'AbNormal': 1}),
        test_size=0.2,
        shuffle=True,
        random_state=RANDOM_STATE,
    )

    # VotingClassifier 설정
    voting_clf = VotingClassifier(estimators=estimators, voting=voting)

    # 모델 학습
    voting_clf.fit(x_train, y_train)

    if voting == 'soft':
        # 소프트 보팅의 경우 확률 예측
        y_val_pred_proba = voting_clf.predict_proba(x_val)[:, 1]
        y_val_pred = (y_val_pred_proba >= threshold).astype(int)
    else:
        # 하드 보팅의 경우 직접 예측
        y_val_pred = voting_clf.predict(x_val)

    # 평가지표 계산
    f1 = f1_score(y_val, y_val_pred, average="binary")
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred, zero_division=0)
    recall = recall_score(y_val, y_val_pred)
    conf_matrix = confusion_matrix(y_val, y_val_pred)
    
    # 결과 출력
    print(f'Voting Classifier로 학습한 결과:')
    print(f'F1 Score: {f1}')
    print('---')
    print('Confusion Matrix:')
    print(conf_matrix)
    print('---')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print('\n')

    return voting_clf  # 학습된 VotingClassifier 반환

def voting(preds_or_probs, method='soft', threshold=0.3):
    """
    하드 보팅 또는 소프트 보팅을 사용하여 최종 예측을 수행합니다.

    Parameters:
    preds_or_probs (list of np.array): 각 모델의 예측 배열 리스트 (하드 보팅) 또는 예측 확률 배열 리스트 (소프트 보팅)
    method (str): 'soft' 또는 'hard' 보팅 방법 선택
    threshold (float): 소프트 보팅 시 예측을 양성으로 간주할 확률 임계값

    Returns:
    np.array: 최종 예측 결과
    """
    if method == 'soft':
        # 소프트 보팅: 각 모델의 확률 평균 계산
        soft_voting_probs = np.mean(preds_or_probs, axis=0)
        # 최종 예측: 평균 확률에 대해 스레드 홀드 적용
        final_predictions = (soft_voting_probs >= threshold).astype(int)
    elif method == 'hard':
        # 하드 보팅: 각 모델의 예측을 모아서 다수결 원칙 적용
        preds = np.array(preds_or_probs)
        final_predictions = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=preds)
    else:
        raise ValueError("method 인자는 'soft' 또는 'hard'여야 합니다.")
    
    return final_predictions

공정별 모델 구축

lightgbm

In [11]:
model_Dam = fit_all_train_data_function(
    'lgbm', train_data_dam
    , n_estimators=2470
    , num_leaves=2454
    , max_depth=26
    , learning_rate=0.06067228197373452
    , min_child_samples=134
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

model_AutoClave = fit_all_train_data_function(
    'lgbm', train_data_autoclave
    , n_estimators=731
    , num_leaves=996
    , max_depth=273
    , learning_rate=0.0912254393922836
    , min_child_samples=195
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

model_Fill1 = fit_all_train_data_function(
    'lgbm', train_data_fill1
    , n_estimators=821
    , num_leaves=1400
    , max_depth=52
    , learning_rate=0.002743887584386348
    , min_child_samples=231
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

model_Fill2 = fit_all_train_data_function(
    'lgbm', train_data_fill2
    , n_estimators=1005
    , num_leaves=2304
    , max_depth=293
    , learning_rate=0.08460539739469425
    , min_child_samples=272
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

# model_All = fit_all_train_data_function(
#     'lgbm', train_data
#     , n_estimators=1496
#     , num_leaves=1611
#     , max_depth=148
#     , learning_rate=0.0822880159816304
#     , min_child_samples=194
#     , boosting_type='dart'
#     , random_state=RANDOM_STATE
#     , verbose=-1
# )


# 예측에 필요한 데이터 분리
x_test_dam = test_data_dam.drop(["target", "Set ID"], axis=1)
x_test_autoclave = test_data_autoclave.drop(["target", "Set ID"], axis=1)
x_test_fill1 = test_data_fill1.drop(["target", "Set ID"], axis=1)
x_test_fill2 = test_data_fill2.drop(["target", "Set ID"], axis=1)
# x_test_all = test_data.drop(["target", "Set ID"], axis=1)

# 예측 확률 리스트 (소프트 보팅용)
lgbm_probs = [
    model_Dam.predict_proba(x_test_dam)[:, 1]
    , model_AutoClave.predict_proba(x_test_autoclave)[:, 1]
    , model_Fill1.predict_proba(x_test_fill1)[:, 1]
    , model_Fill2.predict_proba(x_test_fill2)[:, 1]
#     , model_All.predict_proba(x_test_all)[:, 1]
]

lgbm 모델이 train_data_dam 데이터로 학습 완료
lgbm 모델이 train_data_autoclave 데이터로 학습 완료
lgbm 모델이 train_data_fill1 데이터로 학습 완료
lgbm 모델이 train_data_fill2 데이터로 학습 완료


In [12]:
# 소프트 보팅 결과
final_predictions = voting(lgbm_probs, method='soft', threshold=0.22)
print(sum(final_predictions))

# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("./data/submission.csv")
df_sub["target"] = final_predictions

# df_sub['target'] 값을 문자열 레이블로 변환
df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')

# 제출 파일 저장
df_sub.to_csv("./data/data0827_lgbm(4).csv", index=False)

855


xgb

In [11]:
model_Dam = fit_all_train_data_function(
    'xgb', train_data_dam
    , n_estimators = 1244
    , learning_rate = 0.1258535425769987
    , max_depth = 26
    , alpha = 2.1820842842359597e-06
    , gamma = 0.00010809657684921935
    , reg_alpha = 0.5844029076359536
    , reg_lambda = 0.4748752246073433
    , colsample_bytree = 0.9607659760060685
    , subsample = 0.7147741317935203
    , objective = 'binary:logistic'
    , tree_method = 'exact'
    , random_state=RANDOM_STATE
)

model_AutoClave = fit_all_train_data_function(
    'xgb', train_data_autoclave,
    n_estimators = 1152, 
    learning_rate = 0.02466611382982541, 
    max_depth = 29, 
    alpha = 2.9180083404308157e-05, 
    gamma = 0.00012667501319666823, 
    reg_alpha = 0.6903592486292155, 
    reg_lambda = 0.5638873235014423, 
    colsample_bytree = 0.9432782030604233, 
    subsample = 0.19192246128663584,
    objective = 'binary:logistic',  # 이진 분류
    tree_method = "exact", 
    random_state=RANDOM_STATE
)

model_Fill1 = fit_all_train_data_function(
    'xgb', train_data_fill1,
    n_estimators = 1899, 
    learning_rate = 0.011878583548993711, 
    max_depth = 12, 
    alpha = 0.004515243354832891,
    gamma = 0.0015693650802180896,
    reg_alpha = 0.7484424912256998, 
    reg_lambda = 0.27164326303977143, 
    colsample_bytree = 0.7901385059430825,
    subsample = 0.9924662032617025,
    objective = 'binary:logistic',
    tree_method = 'exact',
    random_state=RANDOM_STATE
)

model_Fill2 = fit_all_train_data_function(
    'xgb', train_data_fill2,
    n_estimators = 1162, 
    learning_rate = 0.014523070494025153, 
    max_depth = 8, 
    alpha = 0.00012198482017902725, 
    gamma = 0.001236902841680112, 
    reg_alpha = 0.7331637000614692, 
    reg_lambda = 0.5237223061096699, 
    colsample_bytree = 0.8250374170841293, 
    subsample = 0.31906427054137687,
    objective = 'binary:logistic',
    tree_method = 'exact',
    random_state=RANDOM_STATE
)

# model_All = fit_all_train_data_function(
#     'xgb', train_data,
#     n_estimators = 2427,
#     learning_rate = 0.010774204513905965, 
#     max_depth = 17, 
#     alpha = 0.0005233654110538582, 
#     gamma = 5.551445919277608e-05, 
#     reg_alpha = 0.9652805882189326, 
#     reg_lambda = 0.3542856398135083, 
#     colsample_bytree = 0.9094884645797131, 
#     subsample = 0.1733751790853043,
#     objective = 'binary:logistic',  # 이진 분류
#     tree_method = "exact", 
#     random_state=RANDOM_STATE
# )

# 예측에 필요한 데이터 분리
x_test_dam = test_data_dam.drop(["target", "Set ID"], axis=1)
x_test_autoclave = test_data_autoclave.drop(["target", "Set ID"], axis=1)
x_test_fill1 = test_data_fill1.drop(["target", "Set ID"], axis=1)
x_test_fill2 = test_data_fill2.drop(["target", "Set ID"], axis=1)
# x_test_all = test_data.drop(["target", "Set ID"], axis=1)


# 예측 확률 리스트 (소프트 보팅용)
xgb_probs = [
    model_Dam.predict_proba(x_test_dam)[:, 1]
    , model_AutoClave.predict_proba(x_test_autoclave)[:, 1]
    , model_Fill1.predict_proba(x_test_fill1)[:, 1]
    , model_Fill2.predict_proba(x_test_fill2)[:, 1]
#     , model_All.predict_proba(x_test_all)[:, 1]
]


xgb 모델이 train_data_dam 데이터로 학습 완료
xgb 모델이 train_data_autoclave 데이터로 학습 완료
xgb 모델이 train_data_fill1 데이터로 학습 완료
xgb 모델이 train_data_fill2 데이터로 학습 완료


In [12]:
# 소프트 보팅 결과
final_predictions = voting(xgb_probs, method='soft', threshold=0.3)
print(sum(final_predictions))

# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("./data/submission.csv")
df_sub["target"] = final_predictions

# df_sub['target'] 값을 문자열 레이블로 변환
df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')

# 제출 파일 저장
df_sub.to_csv("./data/data0827_xgb(4).csv", index=False)

334


cat

In [13]:
model_Dam = fit_all_train_data_function(
    'cat', train_data_dam,
    iterations = 1478, 
    learning_rate = 0.009068953796649421, 
    depth = 11, 
    min_data_in_leaf = 2,
    l2_leaf_reg = 1.187291687951122,
    random_strength = 0.43102541391012816, 
    bagging_temperature = 3.1790702578164853, 
    border_count = 155, 
    scale_pos_weight = 1.4418307437388553,
    grow_policy = 'Depthwise',

    random_state = RANDOM_STATE,
    eval_metric = 'F1',
    logging_level = 'Silent',
    boosting_type = 'Plain'
)

model_AutoClave = fit_all_train_data_function(
    'cat', train_data_autoclave,
    iterations = 1299, 
    learning_rate =  0.03808793470493637, 
    depth = 9, 
    min_data_in_leaf = 5,
    l2_leaf_reg = 4.942829707223811, 
    random_strength = 3.804933757402697, 
    bagging_temperature = 1.3151583440997139, 
    border_count = 286, 
    scale_pos_weight = 1.9749286362629779,
    grow_policy = 'SymmetricTree',

    random_state = RANDOM_STATE,
    eval_metric = 'F1',
    logging_level = 'Silent',
    boosting_type = 'Plain'
)

model_Fill1 = fit_all_train_data_function(
    'cat', train_data_fill1,
    iterations = 2842, 
    learning_rate = 0.01099464761153367, 
    depth = 4, 
    min_data_in_leaf = 3,
    l2_leaf_reg = 3.7373183252945945, 
    random_strength = 9.3675281753561, 
    bagging_temperature = 4.750112155842117, 
    border_count = 160, 
    scale_pos_weight = 2.53860325765727,
    grow_policy = 'Lossguide',

    random_state = RANDOM_STATE,
    eval_metric = 'F1',
    logging_level = 'Silent',
    boosting_type = 'Plain'
)

model_Fill2 = fit_all_train_data_function(
    'cat', train_data_fill2,
    iterations = 1458, 
    learning_rate = 0.004706507801075929, 
    depth = 13, 
    min_data_in_leaf = 4,
    l2_leaf_reg = 1.909987690181427, 
    random_strength = 9.047942432889677, 
    bagging_temperature = 3.545210494821586, 
    border_count = 300, 
    scale_pos_weight = 3.4781865667208467,
    grow_policy = 'Lossguide',


    random_state = RANDOM_STATE,
    eval_metric = 'F1',
    logging_level = 'Silent',
    boosting_type = 'Plain'
)

# model_All = fit_all_train_data_function(
#     'cat', train_data,
#     iterations=1349,
#     learning_rate=0.012526639112437014,
#     depth=9,
#     min_data_in_leaf=4,
#     l2_leaf_reg=2.245006704049574,
#     random_strength=0.6922797458293842,
#     bagging_temperature=8.230635636022027,
#     border_count=211,
#     scale_pos_weight=2.0709015241138236,
#     grow_policy='Depthwise',
    
#     random_state=RANDOM_STATE,
#     eval_metric='F1',
#     logging_level='Silent',
#     boosting_type='Plain'
# )

# 예측에 필요한 데이터 분리
x_test_dam = test_data_dam.drop(["target", "Set ID"], axis=1)
x_test_autoclave = test_data_autoclave.drop(["target", "Set ID"], axis=1)
x_test_fill1 = test_data_fill1.drop(["target", "Set ID"], axis=1)
x_test_fill2 = test_data_fill2.drop(["target", "Set ID"], axis=1)
# x_test_all = test_data.drop(["target", "Set ID"], axis=1)

# 예측 확률 리스트 (소프트 보팅용)
cat_probs = [
    model_Dam.predict_proba(x_test_dam)[:, 1]
    , model_AutoClave.predict_proba(x_test_autoclave)[:, 1]
    , model_Fill1.predict_proba(x_test_fill1)[:, 1]
    , model_Fill2.predict_proba(x_test_fill2)[:, 1]
#     , model_All.predict_proba(x_test_all)[:, 1]
]


cat 모델이 train_data_dam 데이터로 학습 완료
cat 모델이 train_data_autoclave 데이터로 학습 완료
cat 모델이 train_data_fill1 데이터로 학습 완료
cat 모델이 train_data_fill2 데이터로 학습 완료


In [14]:
# 소프트 보팅 결과
final_predictions = voting(cat_probs, method='soft', threshold=0.28)
print(sum(final_predictions))

# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("./data/submission.csv")
df_sub["target"] = final_predictions

# df_sub['target'] 값을 문자열 레이블로 변환
df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')

# 제출 파일 저장
df_sub.to_csv("./data/data0827_cat(4).csv", index=False)

714


et

In [15]:
model_Dam = fit_all_train_data_function(
    'et', train_data_dam,
    n_estimators = 1532,
    max_depth = 25,
    min_samples_split = 3,
    min_samples_leaf = 1,
    criterion = 'entropy',
    bootstrap = False,
    random_state=RANDOM_STATE
)

model_AutoClave = fit_all_train_data_function(
    'et', train_data_autoclave,
    n_estimators = 1943,
    max_depth = 34,
    min_samples_split = 6,
    min_samples_leaf = 1,
    criterion = 'entropy',
    bootstrap = False,
    random_state=RANDOM_STATE
)

model_Fill1 = fit_all_train_data_function(
    'et', train_data_fill1,
    n_estimators = 2040,
    max_depth = 25,
    min_samples_split = 4,
    min_samples_leaf = 1,
    criterion = 'entropy',
    bootstrap = False,
    random_state=RANDOM_STATE
)

model_Fill2 = fit_all_train_data_function(
    'et', train_data_fill2
    , n_estimators = 709
    , max_depth = 44
    , min_samples_split = 3
    , min_samples_leaf = 1
    , criterion = 'entropy'
    , bootstrap = False
    , random_state=RANDOM_STATE
)

# model_All = fit_all_train_data_function(
#     'et', train_data,
#     n_estimators = 1085,
#     max_depth = 30,
#     min_samples_split = 3,
#     min_samples_leaf = 1,
#     criterion = 'entropy',
#     bootstrap = False,
#     random_state=RANDOM_STATE
# )

# 예측에 필요한 데이터 분리
x_test_dam = test_data_dam.drop(["target", "Set ID"], axis=1)
x_test_autoclave = test_data_autoclave.drop(["target", "Set ID"], axis=1)
x_test_fill1 = test_data_fill1.drop(["target", "Set ID"], axis=1)
x_test_fill2 = test_data_fill2.drop(["target", "Set ID"], axis=1)
# x_test_all = test_data.drop(["target", "Set ID"], axis=1)

# 예측 확률 리스트 (소프트 보팅용)
et_probs = [
    model_Dam.predict_proba(x_test_dam)[:, 1]
    , model_AutoClave.predict_proba(x_test_autoclave)[:, 1]
    , model_Fill1.predict_proba(x_test_fill1)[:, 1]
    , model_Fill2.predict_proba(x_test_fill2)[:, 1]
#     , model_All.predict_proba(x_test_all)[:, 1]
]


et 모델이 train_data_dam 데이터로 학습 완료
et 모델이 train_data_autoclave 데이터로 학습 완료
et 모델이 train_data_fill1 데이터로 학습 완료
et 모델이 train_data_fill2 데이터로 학습 완료


In [16]:
# 소프트 보팅 결과
final_predictions = voting(et_probs, method='soft', threshold=0.3)
print(sum(final_predictions))

# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("./data/submission.csv")
df_sub["target"] = final_predictions

# df_sub['target'] 값을 문자열 레이블로 변환
df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')

# 제출 파일 저장
df_sub.to_csv("./data/data0827_et(4).csv", index=False)

344


rf

In [17]:
model_Dam = fit_all_train_data_function(
    'rf', train_data_dam
    , n_estimators = 1113
    , max_depth = 43
    , min_samples_split = 3
    , min_samples_leaf = 1
    , criterion = 'entropy'
    , class_weight = 'balanced'
    , random_state=RANDOM_STATE
)

model_AutoClave = fit_all_train_data_function(
    'rf', train_data_autoclave
    , n_estimators = 2744
    , max_depth = 27
    , min_samples_split = 3
    , min_samples_leaf = 1
    , criterion = 'entropy'
    , class_weight = 'balanced'
    , random_state=RANDOM_STATE
)

model_Fill1 = fit_all_train_data_function(
    'rf', train_data_fill1
    , n_estimators = 1071
    , max_depth = 61
    , min_samples_split = 4
    , min_samples_leaf = 3
    , criterion = 'entropy'
    , class_weight = 'balanced'
    , random_state=RANDOM_STATE
)

model_Fill2 = fit_all_train_data_function(
    'rf', train_data_fill2
    , n_estimators = 2535
    , max_depth = 100
    , min_samples_split = 5
    , min_samples_leaf = 1
    , criterion = 'entropy'
    , class_weight = 'balanced'
    , random_state=RANDOM_STATE
)

# model_All = fit_all_train_data_function(
#     'rf', train_data
#     , n_estimators = 1047
#     , max_depth = 39
#     , min_samples_split = 4
#     , min_samples_leaf = 1
#     , criterion = 'entropy'
#     , class_weight = 'balanced'
#     , random_state=RANDOM_STATE
# )

# 예측에 필요한 데이터 분리
x_test_dam = test_data_dam.drop(["target", "Set ID"], axis=1)
x_test_autoclave = test_data_autoclave.drop(["target", "Set ID"], axis=1)
x_test_fill1 = test_data_fill1.drop(["target", "Set ID"], axis=1)
x_test_fill2 = test_data_fill2.drop(["target", "Set ID"], axis=1)
# x_test_all = test_data.drop(["target", "Set ID"], axis=1)

# 예측 확률 리스트 (소프트 보팅용)
rf_probs = [
    model_Dam.predict_proba(x_test_dam)[:, 1]
    , model_AutoClave.predict_proba(x_test_autoclave)[:, 1]
    , model_Fill1.predict_proba(x_test_fill1)[:, 1]
    , model_Fill2.predict_proba(x_test_fill2)[:, 1]
#     , model_All.predict_proba(x_test_all)[:, 1]
]


rf 모델이 train_data_dam 데이터로 학습 완료
rf 모델이 train_data_autoclave 데이터로 학습 완료
rf 모델이 train_data_fill1 데이터로 학습 완료
rf 모델이 train_data_fill2 데이터로 학습 완료


In [18]:
# 소프트 보팅 결과
final_predictions = voting(rf_probs, method='soft', threshold=0.32)  
print(sum(final_predictions))

# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("./data/submission.csv")
df_sub["target"] = final_predictions

# df_sub['target'] 값을 문자열 레이블로 변환
df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')

# 제출 파일 저장
df_sub.to_csv("./data/data0827_rf(4).csv", index=False)

1090


ada

In [19]:
base_estimator = DecisionTreeClassifier(
    max_depth=27,
    min_samples_split=4,
    min_samples_leaf=1,
    max_features=0.8675781747711193,
    random_state=RANDOM_STATE
)

model_Dam = fit_all_train_data_function(
    'ada', train_data_dam
    , estimator=base_estimator
    , n_estimators=833
    , learning_rate=0.9817802538802249
    , random_state=RANDOM_STATE
)

base_estimator = DecisionTreeClassifier(
    max_depth=15,
    min_samples_split=35,
    min_samples_leaf=2,
    max_features=0.16617258789907924,
    random_state=RANDOM_STATE
)

model_AutoClave = fit_all_train_data_function(
    'ada', train_data_autoclave
    , estimator=base_estimator
    , n_estimators=118
    , learning_rate=0.4047552485696655
    , random_state=RANDOM_STATE
)

base_estimator = DecisionTreeClassifier(
    max_depth=12,
    min_samples_split=27,
    min_samples_leaf=8,
    max_features=0.6030749544065899,
    random_state=RANDOM_STATE
)

model_Fill1 = fit_all_train_data_function(
    'ada', train_data_fill1
    , estimator=base_estimator
    , n_estimators=400
    , learning_rate=0.48125695338518293
    , random_state=RANDOM_STATE
)

base_estimator = DecisionTreeClassifier(
    max_depth=13,
    min_samples_split=4,
    min_samples_leaf=2,
    max_features=0.8686759201566782,
    random_state=RANDOM_STATE
)

model_Fill2 = fit_all_train_data_function(
    'ada', train_data_fill2
    , estimator=base_estimator
    , n_estimators=517
    , learning_rate=0.25710844249446485
    , random_state=RANDOM_STATE
)

# base_estimator = DecisionTreeClassifier(
#     max_depth=6,
#     min_samples_split=30,
#     min_samples_leaf=12,
#     max_features=0.8457354826252363,
#     random_state=RANDOM_STATE
# )

# model_All = fit_all_train_data_function(
#     'ada', train_data
#     , estimator=base_estimator
#     , n_estimators=541
#     , learning_rate=0.3430292944431681
#     , random_state=RANDOM_STATE
# )

# 예측에 필요한 데이터 분리
x_test_dam = test_data_dam.drop(["target", "Set ID"], axis=1)
x_test_autoclave = test_data_autoclave.drop(["target", "Set ID"], axis=1)
x_test_fill1 = test_data_fill1.drop(["target", "Set ID"], axis=1)
x_test_fill2 = test_data_fill2.drop(["target", "Set ID"], axis=1)
# x_test_all = test_data.drop(["target", "Set ID"], axis=1)

# 예측 확률 리스트 (소프트 보팅용)
ada_probs = [
    model_Dam.predict_proba(x_test_dam)[:, 1]
    , model_AutoClave.predict_proba(x_test_autoclave)[:, 1]
    , model_Fill1.predict_proba(x_test_fill1)[:, 1]
    , model_Fill2.predict_proba(x_test_fill2)[:, 1]
#     , train_model_All.predict_proba(x_test_all)[:, 1]
]


ada 모델이 train_data_dam 데이터로 학습 완료
ada 모델이 train_data_autoclave 데이터로 학습 완료
ada 모델이 train_data_fill1 데이터로 학습 완료
ada 모델이 train_data_fill2 데이터로 학습 완료


In [22]:
# 소프트 보팅 결과
final_predictions = voting(ada_probs, method='soft', threshold=0.26)
print(sum(final_predictions))

# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("./data/submission.csv")
df_sub["target"] = final_predictions

# df_sub['target'] 값을 문자열 레이블로 변환
df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')

# 제출 파일 저장
df_sub.to_csv("./data/data0827_ada(4).csv", index=False)

1231


Hard voting

공정구분하여 학습한 개별 모델들에 대해서 hard voting

In [23]:
import pandas as pd
import numpy as np
from collections import Counter

def read_submission_files(file_paths):
    """
    제출 파일을 읽어와서 예측 결과를 반환합니다.

    Parameters:
    file_paths (list of str): 제출 파일 경로 리스트

    Returns:
    list of np.array: 각 제출 파일의 예측 결과 리스트
    """
    predictions = []
    for file_path in file_paths:
        df = pd.read_csv(file_path)
        preds = df['target'].apply(lambda x: 1 if x == 'AbNormal' else 0).values
        predictions.append(preds)
    return predictions

def hard_voting(preds):
    """
    하드 보팅을 사용하여 최종 예측을 수행합니다.

    Parameters:
    preds (list of np.array): 각 모델의 예측 배열 리스트

    Returns:
    np.array: 최종 예측 결과
    """
    preds = np.array(preds)
    
    # 각 샘플의 예측 결과를 문자열로 변환하여 리스트에 저장
    sample_predictions = [''.join(map(str, x)) for x in preds.T]
    
    # 각 예측 결과의 빈도수를 계산
    prediction_counts = Counter(sample_predictions)
    
    # 빈도수 출력
    for pred, count in prediction_counts.items():
        print(f"Prediction {pred}: {count} times")
    
    # 하드 보팅을 통해 최종 예측을 계산
    final_predictions = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=preds)
    return final_predictions

csv 파일로 hard voting 방식

In [24]:
import pandas as pd

# 공통 경로
common_path = "./data/"

# 제출 파일 이름 리스트
file_names = [
    "data0827_lgbm(4).csv"
    , "data0827_lgbm(4).csv"
    , "data0827_xgb(4).csv"
    , "data0827_xgb(4).csv"
    , "data0827_cat(4).csv"
    , "data0827_cat(4).csv"
    , "data0827_et(4).csv"
    , "data0827_rf(4).csv"
    , "data0827_ada(4).csv"   
    # 파일 추가 가능  <----- 파일 필요시 추가하세요!!
]

# 경로를 추가하는 함수
def add_common_path(file_names, common_path):
    return [common_path + file_name for file_name in file_names]

# 경로가 추가된 파일 리스트
file_paths = add_common_path(file_names, common_path)

# 제출 파일에서 예측 결과 읽어오기
predictions = read_submission_files(file_paths)

# 하드 보팅 결과
final_predictions_hard = hard_voting(predictions)

# 결과를 새로운 제출 파일로 저장할 파일 이름
output_file_name = "model_name_change_submission.csv" # <----- 파일 이름을 변경하세요!!

# 결과를 새로운 제출 파일로 저장
df_sub = pd.read_csv(file_paths[0])
df_sub["target"] = final_predictions_hard
df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')
df_sub.to_csv(output_file_name, index=False)

print(f"최종 제출 파일이 '{output_file_name}'로 저장되었습니다.")

FileNotFoundError: [Errno 2] No such file or directory: './data/data0827_xgb(4).csv'

In [ ]:
df_sub['target'].value_counts()

In [159]:
df_sub.head(10)

,Set ID,target
0,0001be084fbc4aaa9d921f39e595961b,Normal
1,0005bbd180064abd99e63f9ed3e1ac80,Normal
2,000948934c4140d883d670adcb609584,Normal
3,000a6bfd02874c6296dc7b2e9c5678a7,Normal
4,0018e78ce91343678716e2ea27a51c95,Normal
5,001fda4596f545d0a3b0ce85fbea77d2,Normal
6,0020734a7b29472298358ad58645a0c9,Normal
7,00234c5914cd4c4a888d13f8b3773135,Normal
8,00297b6c93e44d49ac534758a23dc74e,AbNormal
9,002d904240d84b188d410d16383a9c3a,Normal


우측 상단의 제출 버튼을 클릭해 결과를 확인하세요

.